# Calcite solubility in water and CO<sub>2</sub>-saturated rainwater

The anthropogenic CO<sub>2</sub> absorbed by the oceans results in an increase in the concentration of hydrogen ions (H<sup>+</sup>) and in bicarbonate ions (HCO<sub>3</sub><sup>-</sup>) and a decrease in carbonate ions (CO<sub>3</sub><sup>-2</sup>). A review article of *Figuerola et al., A Review and Meta-Analysis of Potential Impacts of Ocean Acidification on Marine Calcifiers From the Southern Ocean, Front. Mar. Sci., 2021* identifies *ocean acidification (OA)* (accompanied by reduction in CO<sub>3</sub><sup>-2</sup>, see figure below) as a critical issue for shells and skeletons of marine calcifiers such as foraminifera, corals, echinoderms, molluscs, and bryozoans. OA process is shallowing the carbonate saturation horizon (which is the depth below which calcium carbonate dissolves) likely increasing the vulnerability of many resident marine calcifiers to dissolution. Moreover, ocean warming could further exacerbate the effects of OA in these particular species. That is why an understanding of the dependence of calcite solubility on various factors is an important modelling question.

|![Infographic of the ocean acidification process](../../images/ocian-acidification.jpeg)|
|:--:|
|Infographic of the ocean acidification process, Source: frontiersin.org|

In this tutorial, we investigate the dependence of calcite solubility in water (closed system) and carbon-dioxide saturated rainwater (open system) on temperature and pressure change. We also study calcite solubility trends, when adding carbon dioxide into the system.

First, we initialize the chemical system with aqueous, gaseous, and calcite phases.

In [ ]:
from reaktoro import *
import numpy as np
import pandas as pd

# Create the database
db = SupcrtDatabase("supcrtbl")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("H O C Ca Mg K Cl Na S N"))
aqueousphase.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2"),
))

# Create a gaseous phase
gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.setActivityModel(ActivityModelPengRobinson())

# Create a mineral phase
mineral = MineralPhase("Calcite")

# Create the chemical system
system = ChemicalSystem(db, aqueousphase, gaseousphase, mineral)

Next, we set up equilibrium specifications, equilibrium conditions, and equilibrium solver all to be used for the equilibrium calculations. To constrain the charge of the chemical state, we need to make it open to the Cl<sup>-</sup>. Finally, we create aqueous properties to evaluate pH in the forthcoming calculations.

In [ ]:
# Define equilibrium specs
specs = EquilibriumSpecs (system)
specs.temperature()
specs.pressure()
specs.charge()
specs.openTo("Cl-")

# Define equilibrium conditions to be satisfied at chemical equilibrium
conditions = EquilibriumConditions(specs)
conditions.charge(0.0, "mol") # to make sure the mixture is neutral

# Define equilibrium solver
solver = EquilibriumSolver(specs)

# Define aqueous properties
aprops = AqueousProps(system)

Functions below define the chemical states corresponding to the pure water (also referred to as a closed system), rainwater saturated with carbon dioxide, and seawater, respectively:

In [ ]:
def water():

    state = ChemicalState(system)
    state.add("H2O(aq)", 1.0, "kg")

    return state

def rainwater():

    state = ChemicalState(system)
    # Rainwater composition
    state.set("H2O(aq)", 1.0, "kg")
    state.set("Na+"    , 2.05, "mg") # Sodium, 2.05 ppm = 2.05 mg/L ~ 2.05 mg/kgw
    state.set("K+"     , 0.35, "mg") # Potassium
    state.set("Ca+2"   , 1.42, "mg") # Calcium
    state.set("Mg+2"   , 0.39, "mg") # Magnesium
    state.set("Cl-"    , 3.47, "mg") # Chloride
    state.set("SO4-2"  , 2.19, "mg")
    state.set("NO3-"   , 0.27, "mg")
    state.set("NH4+"   , 0.41, "mg")
    state.set("CO2(aq)", 0.36, "mol")  # rainwater is saturated with CO2

    return state

def seawater():

    state = ChemicalState(system)
    # Seawater composition
    state.setTemperature(25, "celsius")
    state.setPressure(1.0, "bar")
    state.add("H2O(aq)",     1.0, "kg")
    state.add("Ca+2"   ,   412.3, "mg")
    state.add("Mg+2"   ,  1290.0, "mg")
    state.add("Na+"    , 10768.0, "mg")
    state.add("K+"     ,   399.1, "mg")
    state.add("Cl-"    , 19353.0, "mg")
    state.add("HCO3-"  ,   141.7, "mg")
    state.add("SO4-2"  ,  2712.0, "mg")

    return state

## Solubility of calcite for different temperatures and pressures

Next, we initialize the array of temperatures from 20 &deg;C till 90 &deg;C and calculate solubilities of calcite in water and CO<sub>2</sub>-saturated rainwater for pressures P = 1, 10, 100 bar.

In [ ]:
temperatures = np.arange(20.0, 91.0, 5.0)   # in celsius
pressures = np.array([1, 10, 100])          # in bar

df_water = pd.DataFrame(columns=["T", "P", "pH", "deltaCalcite"])
df_rainwater = pd.DataFrame(columns=["T", "P", "pH", "deltaCalcite"])

def solubility_of_calcite(state, T, P, tag):

    # Initial amount of calcite
    n0Calcite = 10.0

    conditions.temperature(T, "celsius")
    conditions.pressure(P, "bar")

    # Equilibrate the solution given by the chemical state and conditions
    res = solver.solve(state, conditions)

    # Add `n0Calcite` amount of calcite
    state.set("Calcite", n0Calcite, "mol")

    # Equilibrate solution with added calcite
    res = solver.solve(state)

    # Update aqueous properties
    aprops.update(state)

    # Fetch the amount of final calcite in the equilibrium state
    nCalcite = float(state.speciesAmount("Calcite"))

    if tag == "water":
        df_water.loc[len(df_water)] = [T, P, float(aprops.pH()), n0Calcite - nCalcite]
    elif tag == "rainwater":
        df_rainwater.loc[len(df_rainwater)] = [T, P, float(aprops.pH()), n0Calcite - nCalcite]

for T in temperatures:
    for P in pressures:
        solubility_of_calcite(water(), T, P, "water")
        solubility_of_calcite(rainwater(), T, P, "rainwater")

Let us check if the obtained value of solubility for 25 &deg;C and 1 bar indeed corresponds to the values of Wikipedia, i.e., the solubility in water equals to 0.013 g/L (25 &deg;C). Below, we use 100.0869 g/mol as the calcite molar mass:

In [ ]:
df_water_25C = df_water[df_water['T'] == 25.0] # fetch data corresponding to T = 25 celsius
df_water_25C_P1 = df_water_25C[df_water_25C['P'] == 1.0] # fetch data corresponding to T = 25 celsius and P = 1 bar
deltaCalcite = df_water_25C_P1['deltaCalcite'].iloc[0]
print(f"Solubility of calcite in water (closed system) equals to {deltaCalcite:.6f} "
      f"mol/kgw = ... = {deltaCalcite * 0.1000869 * 1e3:.6f} g/L")

Thus, in the closed system (with pure water), approximately 0.116 mmol of calcite dissolve. The amount of calcite that dissolves is independent of the initial value (provided that it exceeds the solubility limit).

We plot using [bokeh](https://bokeh.org/) python library. We start from solubilities of calcite in water and CO<sub>2</sub>-saturated rainwater for pressure P = 1:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

df_water_P1 = df_water[df_water['P'] == 1.0]
df_rainwater_P1 = df_rainwater[df_rainwater['P'] == 1.0]

p = figure(
    title="COMPARISON OF CALCITE SOLUBILITY IN WATER AND RAINWATER",
    x_axis_label=r'TEMPERATURE [°C]',
    y_axis_label='CALCITE SOLUBILITY [MOL/KG]',
    sizing_mode="scale_width",
    plot_height=300)

p.line("T", "deltaCalcite", line_width=3, line_cap="round", line_color='indigo', source=df_water_P1)
p.line("T", "deltaCalcite", line_width=3, line_cap="round", line_color='darkred', source=df_rainwater_P1)

show(p)

The plot illustrates that calcium carbonate has very low solubility in pure water, but in rainwater saturated with carbon dioxide, its solubility increases due to the formation of more soluble calcium bicarbonate. Below, we plot solubilities on the different scales and for different pressure. We see below that increasing pressure also increases the solubility of calcium carbonate.

In [ ]:
from bokeh.models import HoverTool, Legend
from bokeh.plotting import gridplot
from bokeh.models import ColumnDataSource
colors = ['teal', 'darkred', 'indigo', 'coral']

# ----------------------------------- #
# Plot calcite solubility in water
# ----------------------------------- #
hovertool1 = HoverTool()
hovertool1.tooltips = [("T", "@T °C"),
                      ("P", "@P bar"),
                      ("delta(Calcite)", "@deltaCalcite mol")]
p1 = figure(
    title="CALCITE SOLUBILITY IN WATER",
    x_axis_label=r'TEMPERATURE [°C]',
    y_axis_label='CALCITE SOLUBILITY [MOL/KG]',
    sizing_mode="scale_width",
    plot_height=300)

p1.add_tools(hovertool1)

for P, color in zip(pressures, colors):
    df_water_P = ColumnDataSource(df_water[df_water['P'] == P])
    p1.line("T", "deltaCalcite", legend_label=f'P = {P}', line_width=3, line_cap="round", line_color=color, source=df_water_P)

p1.legend.location = 'top_right'

# ----------------------------------- #
# Plot calcite solubility in rainwater
# ----------------------------------- #
hovertool2 = HoverTool()
hovertool2.tooltips = [("T", "@T °C"),
                      ("P", "@P bar"),
                      ("delta(Calcite)", "@deltaCalcite mol")]

p2 = figure(
    title="CALCITE SOLUBILITY IN RAINWATER",
    x_axis_label=r'TEMPERATURE [°C]',
    y_axis_label='CALCITE SOLUBILITY [MOL/KG]',
    sizing_mode="scale_width",
    plot_height=300)

p2.add_tools(hovertool2)

for P, color in zip(pressures, colors):
    df_rainwater_P = ColumnDataSource(df_rainwater[df_water['P'] == P])
    p2.line("T", "deltaCalcite", legend_label=f'P = {P}', line_width=3, line_cap="round", line_color=color, source=df_rainwater_P)

grid = gridplot([[p1], [p2]])

show(grid)

## Solubility of calcite in the seawater with increasing levels of CO<sub>2</sub>

Below, we define the auxiliary variables with initial values of CO<sub>2</sub> amounts and its increment. We run the loop with `nsteps` steps adding CO<sub>2</sub> into seawater and re-equilibrate. The level of added carbon dioxide as well as corresponding pH, we collect into dataframe.

In [ ]:
co2_0 = 0.0
co2_delta = 2.0 # in mmol
nsteps = 50

df_seawater = pd.DataFrame(columns=["amountCO2", "pH", "deltaCalcite", "mCO2aq"])

T = 25
P = 1

for i in range(nsteps):

    # Initial amount of calcite
    n0Calcite = 10.0

    conditions.temperature(T, "celsius")
    conditions.pressure(P, "bar")

    # Add more CO2 to the problem
    state = seawater()
    state.set("CO2(g)", co2_0, "mmol")

    # Equilibrate the solution given by the chemical state and conditions
    res = solver.solve(state, conditions)

    # Add `n0Calcite` amount of calcite
    state.set("Calcite", n0Calcite, "mol")

    # Equilibrate solution with added calcite
    res = solver.solve(state)

    # Update aqueous properties
    aprops.update(state)

    # Fetch the amount of final calcite and CO2(aq) in the equilibrium state
    nCalcite = float(state.speciesAmount("Calcite"))
    nCO2aq   = float(state.speciesAmount("CO2(aq)"))

    # Update CO2 amount
    co2_0 += co2_delta

    # Append new calculated value to the dataframe
    df_seawater.loc[len(df_seawater)] = [co2_0, float(aprops.pH()), n0Calcite - nCalcite, nCO2aq]

Below, we plot pH and calcite solubility dependence on the added CO<sub>2</sub> into seawater.

In [ ]:
hovertool1 = HoverTool()
hovertool1.tooltips = [("amount(CO2)", "@amountCO2 mmol"),
                       ("pH", "@pH"),
                       ("delta(Calcite)", "@deltaCalcite"),
                       ("amount(CO2(aq))", "@mCO2aq")]

p1 = figure(
    title="PH DEPENDENCE ON AMOUNT OF ADDED CO2 TO THE SEAWATER",
    x_axis_label=r'CO2 AMOUNT [MMOL]',
    y_axis_label='PH [-]',
    sizing_mode="scale_width",
    plot_height=300)

p1.add_tools(hovertool1)

p1.line("amountCO2", "pH", line_width=3, line_cap="round", line_color='indigo', source=df_seawater)

p2 = figure(
    title="CALCITE SOLUBILITY DEPENDENCE ON AMOUNT OF ADDED CO2 TO THE SEAWATER",
    x_axis_label=r'CO2 AMOUNT [MMOL]',
    y_axis_label='CALCITE SOLUBILITY [MOL]',
    sizing_mode="scale_width",
    plot_height=300)

p2.add_tools(hovertool1)

p2.line("amountCO2", "deltaCalcite", line_width=3, line_cap="round", line_color='coral', source=df_seawater)

p3 = figure(
    title="AMOUNT OF DISSOLVED CO2 IN THE SEAWATER",
    x_axis_label=r'CO2 AMOUNT [MMOL]',
    y_axis_label='AMOUNT OF CO2(AQ) [MOL]',
    sizing_mode="scale_width",
    plot_height=300)

p3.add_tools(hovertool1)

p3.line("amountCO2", "mCO2aq", line_width=3, line_cap="round", line_color='darkgreen', source=df_seawater)

grid = gridplot([[p1], [p2], [p3]])

show(grid)

Below 45 mmol of CO<sub>2</sub>, the addition of the gas continues making seawater more acidic and therefore increasing the solubility of calcite. However, after reaching the point of saturation, no more gas can be dissolved in seawater and pH remains constant. The solubility of calcite from this point on is also constant.